In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
# Read the data
X = pd.read_csv('../input/titanic/train.csv')
X_test_full = pd.read_csv('../input/titanic/test.csv')

In [ ]:
X.dropna(axis=0, subset=['Survived'], inplace=True)
y = X.Survived              
X.drop(['Name','Ticket','Cabin','Survived','PassengerId'], axis=1, inplace=True)

In [ ]:
y=y.values

In [ ]:
X=X.iloc[:,:].values
X_test_full=X_test_full.iloc[:,:].values

In [ ]:
X_test=X_test_full[:,[1,3,4,5,6,8,10]]

In [ ]:
X[:2,:]

In [ ]:
X_test[:10,:]

In [ ]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy = 'most_frequent')
X=imputer.fit_transform(X)
X_test=imputer.transform(X_test)

In [ ]:
from sklearn.preprocessing import LabelEncoder
lebenc=LabelEncoder()

In [ ]:
arr_new=lebenc.fit_transform(X[:,1:2]).reshape(891,1)

In [ ]:
X[:,1:2]=arr_new

In [ ]:
X_test[:,1:2]=lebenc.transform(X_test[:,1:2]).reshape(418,1)

In [ ]:
from sklearn.preprocessing import OneHotEncoder

In [ ]:
ohe=OneHotEncoder(handle_unknown='ignore',sparse=False)
X_new=ohe.fit_transform(X[:,6:7])
X_test_new=ohe.transform(X_test[:,6:7])

In [ ]:
X=X[:,:6]
X_test=X_test[:,:6]

In [ ]:
X_test

In [ ]:
X_new[:,]

In [ ]:
X

In [ ]:
X=np.concatenate((X,X_new),axis=1)
X_test=np.concatenate((X_test,X_test_new),axis=1)

In [ ]:
from xgboost import XGBRegressor
# Define the model
my_model_1 = XGBRegressor(n_estimators=1000,learning_rate=0.05,random_state=0)

# Fit the model
my_model_1.fit(X,y)

In [ ]:
from sklearn.metrics import mean_absolute_error

# Get predictions
predictions_1 = my_model_1.predict(X_test)


In [ ]:
predictions_1=(predictions_1>0.5)

In [ ]:
predictions_1=predictions_1*1

In [ ]:
predictions_1=predictions_1.reshape(418,1)

In [ ]:
predictions_1=np.concatenate((X_test_full[:,0:1],predictions_1[:,0:1]),axis=1)

In [ ]:
# save numpy array as csv file
from numpy import asarray
from numpy import savetxt
# define data
data = asarray(predictions_1)
# save to csv file
savetxt('data.csv', data, delimiter=',')

In [ ]:
from sklearn.ensemble import RandomForestClassifier
random_forest = RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=5, min_samples_split=16,
                       min_weight_fraction_leaf=0.0, n_estimators=700,
                       n_jobs=-1, oob_score=True, random_state=1, verbose=0,
                       warm_start=False)
random_forest.fit(X, y)

In [ ]:
predictions_2=random_forest.predict(X_test)

In [ ]:
predictions_2=predictions_2.reshape(418,1)
predictions_2=np.concatenate((X_test_full[:,0:1],predictions_2[:,0:1]),axis=1)

In [ ]:
# save numpy array as csv file
from numpy import asarray
from numpy import savetxt
# define data
data = asarray(predictions_2)
# save to csv file
savetxt('data_rf.csv', data, delimiter=',')

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_p = sc.fit_transform(X)
X_test_p = sc.transform(X_test)

In [ ]:
# Prediction Using ANN
import keras
from keras.models import Sequential
from keras.layers import Dense


In [ ]:
# Initialising the ANN
classifier = Sequential()

# Adding the input layer and the first hidden layer
classifier.add(Dense(units = 5, kernel_initializer = 'uniform', activation = 'relu', input_dim = 9))

# Adding the second hidden layer
classifier.add(Dense(units = 5, kernel_initializer = 'uniform', activation = 'relu'))

# Adding the output layer
classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))

# Compiling the ANN
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

# Fitting the ANN to the Training set
classifier.fit(X_p, y, batch_size = 10, epochs = 100)

In [ ]:
# Part 3 - Making predictions and evaluating the model

# Predicting the Test set results
y_pred = classifier.predict(X_test_p)
y_pred = (y_pred > 0.5)


In [ ]:
y_pred=y_pred*1
y_pred
predictions_3=np.concatenate((X_test_full[:,0:1],y_pred[:,0:1]),axis=1)

In [ ]:
# save numpy array as csv file
from numpy import asarray
from numpy import savetxt
# define data
data = asarray(predictions_3)
# save to csv file
savetxt('data_ann.csv', data, delimiter=',')